In [1]:
import os
os.chdir('../../' )
cwd=os.getcwd()

from oligo_designer_toolsuite.oligo_specificity_filter._filter_base import ProbeFilterBase
from oligo_designer_toolsuite.oligo_specificity_filter._filter_exact import ProbeFilterExact
from oligo_designer_toolsuite.oligo_specificity_filter._filter_blastn import ProbeFilterBlastn
from oligo_designer_toolsuite.oligo_specificity_filter._filter_bowtie import ProbeFilterBowtie

In [2]:
cwd

'/home/chels/CRISPR/repos/oligo-designer-toolsuite'

In [2]:
n_jobs=10
ligation_region=0
dir_output=cwd + "/output/"
dir_annotations=dir_output+"annotations/"
file_probe_info=dir_annotations + "oligo_DB_unknown_unknown_Custom_release_unknown_gene_transcript"
genes= ['WASH7P', 'DDX11L1', 'TRNT', 'NOC2L', 'PLEKHN1', 'AGRN','UBE2J2', 'DVL1', 'MIB2', 'LOC112268402_1' ]
min_probes_per_gene=2
file_transcriptome_fasta=dir_output +"annotations/reference_DB_unknown_unknown_Custom_release_unknown_genome_False_gene_transcript_True"
word_size=10
percent_identity=80
probe_length_min=30
probe_length_max=40
coverage=50


Test exact match filter

In [3]:
filter_exact=ProbeFilterExact(
        n_jobs,
        dir_output,
        file_probe_info,
        genes,
        dir_annotations)

In [4]:
filter_exact.apply()

INFO:filter_probes:Creating batches
INFO:filter_probes:Exact matches filtered in 2.648837500004447 seconds


Test Blast filter

In [5]:
blast_filter=ProbeFilterBlastn(n_jobs,
        file_transcriptome_fasta, 
        dir_output,
        file_probe_info,
        genes, word_size,
        percent_identity, coverage, probe_length_min, probe_length_max, ligation_region)

In [6]:
blast_filter.apply()

INFO:filter_probes:Creating blast database
INFO:filter_probes:Blast database created in 45.71521049999865 seconds
INFO:filter_probes:Blast alignmnet search finished in 15.541030399996089 seconds
INFO:filter_probes:Blast results processed in 0.9551006999972742 seconds


Test Bowtie filter

In [8]:
n_jobs=10
min_mismatches=4
mismatch_region=5

bowtie_filter=ProbeFilterBowtie(n_jobs,
        dir_output,
        file_probe_info,
        genes,
        probe_length_min,
        probe_length_max,
        file_transcriptome_fasta, min_mismatches, mismatch_region)



In [9]:
bowtie_filter.apply()



# reads processed: 24
# reads with at least one reported alignment: 24 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 24 alignments to 1 output stream(s)
# reads processed: 294
# reads with at least one reported alignment: 294 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 337 alignments to 1 output stream(s)
# reads processed: 307
# reads with at least one reported alignment: 307 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 2967 alignments to 1 output stream(s)
# reads processed: 671
# reads with at least one reported alignment: 671 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 6752 alignments to 1 output stream(s)
# reads processed: 546
# reads with at least one reported alignment: 546 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 1046 alignments to 1 output stream(s)
# reads processed: 339
# reads with at least one reported alignment: 339 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 501 alignments to 1 out

INFO:filter_probes:Bowtie alignmnet search finished in 125.27253470000142 seconds


# reads processed: 1236
# reads with at least one reported alignment: 1236 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 3735 alignments to 1 output stream(s)


INFO:filter_probes:Bowtie results processed in 0.5550013999963994 seconds
